Подключение google disk для использования dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Установка библиотеки

In [2]:
!pip install petroscope

Импорт неободимых библиотек

In [3]:
from pathlib import Path
from petroscope.segmentation.classes import ClassSet, LumenStoneClasses
from petroscope.segmentation.utils import load_image, load_mask
from petroscope.segmentation import GeoSegmModel
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from skimage.color import rgb2lab

Путь к dataset

In [4]:
ds_path = Path('/content/drive/MyDrive/PhotoSets/')

Инциализация набора классов для сегментации

In [5]:
classset = LumenStoneClasses.S1v1()

for cl in classset.classes:
    print(cl)

[0, bg (background), color: #000000]
[1, ccp/kub (chalcopyrite/cubanite), color: #ffa500]
[2, gl (galena), color: #9acd32]
[4, brt (bornite), color: #00bfff]
[6, py/mrc (pyrite/marcasite), color: #2f4f4f]
[8, sph (sphalerite), color: #ee82ee]
[11, tnt/ttr (tenantite/tetrahedrite), color: #483d8b]


Формирование путей к изображениям и маскам

In [6]:
train_img_mask_p = [
    (img_p, ds_path / "masks" / "train" / f"{img_p.stem}.png")
    for img_p in sorted((ds_path / "imgs" / "train").iterdir())
]

test_img_mask_p = [
    (img_p, ds_path / "masks" / "test" / f"{img_p.stem}.png")
    for img_p in sorted((ds_path / "imgs" / "test").iterdir())
]

Загрузка и конвертация обучающих изображений в пространство LAB

In [7]:
for img_p, _ in train_img_mask_p:
    img = load_image(img_p, normalize=True)
    img_lab = rgb2lab(img)
    print(f"Image {img_p.name}: {img_lab.shape}, {img_lab.dtype}")

Image train_01.jpg: (2547, 3396, 3), float32
Image train_02.jpg: (2547, 3396, 3), float32
Image train_03.jpg: (2547, 3396, 3), float32
Image train_04.jpg: (2547, 3396, 3), float32
Image train_06.jpg: (2547, 3396, 3), float32
Image train_07.jpg: (2547, 3396, 3), float32
Image train_08.jpg: (2547, 3396, 3), float32
Image train_09.jpg: (2547, 3396, 3), float32
Image train_10.jpg: (2547, 3396, 3), float32
Image train_11.jpg: (2547, 3396, 3), float32
Image train_12.jpg: (2547, 3396, 3), float32
Image train_13.jpg: (2547, 3396, 3), float32
Image train_14.jpg: (2547, 3396, 3), float32
Image train_15.jpg: (2547, 3396, 3), float32
Image train_16.jpg: (2547, 3396, 3), float32
Image train_17.jpg: (2547, 3396, 3), float32
Image train_18.jpg: (2547, 3396, 3), float32
Image train_19.jpg: (2547, 3396, 3), float32
Image train_20.jpg: (2547, 3396, 3), float32
Image train_21.jpg: (2547, 3396, 3), float32
Image train_22.jpg: (2547, 3396, 3), float32
Image train_23.jpg: (2547, 3396, 3), float32
Image trai

Загрузка масок без one-hot кодирования

In [8]:
for _, mask_p in train_img_mask_p:
    mask = load_mask(mask_p, classes=classset, one_hot=False)
    print(f"Mask {mask_p.name}: {mask.shape}, {mask.dtype}")

Mask train_01.png: (2547, 3396), uint8
Mask train_02.png: (2547, 3396), uint8
Mask train_03.png: (2547, 3396), uint8
Mask train_04.png: (2547, 3396), uint8
Mask train_06.png: (2547, 3396), uint8
Mask train_07.png: (2547, 3396), uint8
Mask train_08.png: (2547, 3396), uint8
Mask train_09.png: (2547, 3396), uint8
Mask train_10.png: (2547, 3396), uint8
Mask train_11.png: (2547, 3396), uint8
Mask train_12.png: (2547, 3396), uint8
Mask train_13.png: (2547, 3396), uint8
Mask train_14.png: (2547, 3396), uint8
Mask train_15.png: (2547, 3396), uint8
Mask train_16.png: (2547, 3396), uint8
Mask train_17.png: (2547, 3396), uint8
Mask train_18.png: (2547, 3396), uint8
Mask train_19.png: (2547, 3396), uint8
Mask train_20.png: (2547, 3396), uint8
Mask train_21.png: (2547, 3396), uint8
Mask train_22.png: (2547, 3396), uint8
Mask train_23.png: (2547, 3396), uint8
Mask train_24.png: (2547, 3396), uint8
Mask train_25.png: (2547, 3396), uint8
Mask train_26.png: (2547, 3396), uint8
Mask train_27.png: (2547,

Загрузка масок с one-hot кодированием

In [9]:
for _, mask_p in train_img_mask_p:
    mask_one_hot = load_mask(mask_p, classes=classset, one_hot=True)
    print(f"Mask one-hot {mask_p.name}: {mask_one_hot.shape}, {mask_one_hot.dtype}")

Mask one-hot train_01.png: (2547, 3396, 7), float32
Mask one-hot train_02.png: (2547, 3396, 7), float32
Mask one-hot train_03.png: (2547, 3396, 7), float32
Mask one-hot train_04.png: (2547, 3396, 7), float32
Mask one-hot train_06.png: (2547, 3396, 7), float32
Mask one-hot train_07.png: (2547, 3396, 7), float32
Mask one-hot train_08.png: (2547, 3396, 7), float32
Mask one-hot train_09.png: (2547, 3396, 7), float32
Mask one-hot train_10.png: (2547, 3396, 7), float32
Mask one-hot train_11.png: (2547, 3396, 7), float32
Mask one-hot train_12.png: (2547, 3396, 7), float32
Mask one-hot train_13.png: (2547, 3396, 7), float32
Mask one-hot train_14.png: (2547, 3396, 7), float32
Mask one-hot train_15.png: (2547, 3396, 7), float32
Mask one-hot train_16.png: (2547, 3396, 7), float32
Mask one-hot train_17.png: (2547, 3396, 7), float32
Mask one-hot train_18.png: (2547, 3396, 7), float32
Mask one-hot train_19.png: (2547, 3396, 7), float32
Mask one-hot train_20.png: (2547, 3396, 7), float32
Mask one-hot

Загрузка цветных масок

In [10]:
for img_p, _ in train_img_mask_p:
    mask_colored_path = ds_path / "masks_colored_png" / "train" / f"{img_p.stem}.png"
    mask_colored = load_image(mask_colored_path, normalize=False)
    print(f"Colored mask {mask_colored_path.name}: {mask_colored.shape}, {mask_colored.dtype}")

Colored mask train_01.png: (2547, 3396, 3), uint8
Colored mask train_02.png: (2547, 3396, 3), uint8
Colored mask train_03.png: (2547, 3396, 3), uint8
Colored mask train_04.png: (2547, 3396, 3), uint8
Colored mask train_06.png: (2547, 3396, 3), uint8
Colored mask train_07.png: (2547, 3396, 3), uint8
Colored mask train_08.png: (2547, 3396, 3), uint8
Colored mask train_09.png: (2547, 3396, 3), uint8
Colored mask train_10.png: (2547, 3396, 3), uint8
Colored mask train_11.png: (2547, 3396, 3), uint8
Colored mask train_12.png: (2547, 3396, 3), uint8
Colored mask train_13.png: (2547, 3396, 3), uint8
Colored mask train_14.png: (2547, 3396, 3), uint8
Colored mask train_15.png: (2547, 3396, 3), uint8
Colored mask train_16.png: (2547, 3396, 3), uint8
Colored mask train_17.png: (2547, 3396, 3), uint8
Colored mask train_18.png: (2547, 3396, 3), uint8
Colored mask train_19.png: (2547, 3396, 3), uint8
Colored mask train_20.png: (2547, 3396, 3), uint8
Colored mask train_21.png: (2547, 3396, 3), uint8


Функция для загрузки и предобработки изображений и масок

In [11]:
def load_and_preprocess(img_path, mask_path, classes, img_size=(256, 256)):
    img = load_image(img_path, normalize=True)
    img = tf.image.resize(img, img_size)

    mask = load_mask(mask_path, classes=classes, one_hot=False)
    mask = tf.image.resize(mask[..., np.newaxis], img_size, method='nearest')
    mask = to_categorical(mask, num_classes=len(classes))

    return img, mask

Модель U-Net

In [12]:
def unet_model(input_size=(256, 256, 3), num_classes=7):
    inputs = Input(input_size)

    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)

    up5 = concatenate([UpSampling2D(size=(2, 2))(conv4), conv3], axis=-1)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(up5)
    conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)
    up6 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(up6)
    conv6 = Conv2D(128, 3, activation='relu', padding='same')(conv6)
    up7 = concatenate([UpSampling2D(size=(2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(up7)
    conv7 = Conv2D(64, 3, activation='relu', padding='same')(conv7)

    outputs = Conv2D(num_classes, 1, activation='softmax')(conv7)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Загрузка и предобработка обучающих данных

In [13]:
train_data = [load_and_preprocess(img_p, mask_p, classset) for img_p, mask_p in train_img_mask_p]
train_images, train_masks = zip(*train_data)
train_images = np.array(train_images)
train_masks = np.array(train_masks)

print(f"Train images shape: {train_images.shape}")
print(f"Train masks shape: {train_masks.shape}")

model = unet_model(num_classes=len(classset.classes))

model.fit([train_images], [train_masks], batch_size=4, epochs=200, validation_split=0.1)

Train images shape: (58, 256, 256, 3)
Train masks shape: (58, 256, 256, 7)
Epoch 1/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.2604 - loss: 2.7425 - val_accuracy: 0.0320 - val_loss: 2.8744
Epoch 2/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 41s 193ms/step - accuracy: 0.3397 - loss: 1.5977 - val_accuracy: 0.3421 - val_loss: 2.1169
Epoch 3/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 201ms/step - accuracy: 0.3767 - loss: 1.4430 - val_accuracy: 0.4005 - val_loss: 2.4244
Epoch 4/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 198ms/step - accuracy: 0.5162 - loss: 1.2350 - val_accuracy: 0.4877 - val_loss: 1.9759
Epoch 5/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 194ms/step - accuracy: 0.6480 - loss: 1.0721 - val_accuracy: 0.4814 - val_loss: 2.2953
Epoch 6/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 201ms/step - accuracy: 0.7185 - loss: 0.9932 - val_accuracy: 0.4951 - val_loss: 2.1782
Epoch 7/200
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 200ms/step - accuracy: 0.7175 - loss: 0.9665 - val_accuracy: 0.4947 - val_loss: 2.3204
Epoch 8/200
13/13 ━━━━━

Тестироание

In [14]:
from petroscope.segmentation.eval import SegmDetailedTester
from tensorflow.keras.utils import to_categorical

tester = SegmDetailedTester(
    Path("output"),
    classes=classset,
    void_pad=0,
    void_border_width=4,
    vis_plots=False,
    vis_segmentation=True,
)

for img_p, mask_p in test_img_mask_p:
    img = load_image(img_p, normalize=True)
    img_resized = tf.image.resize(img, (256, 256))

    pred = model.predict(np.expand_dims(img_resized, axis=0))
    pred = np.argmax(pred[0], axis=-1)
    pred = pred.astype(np.uint8)
    print(f"Pred shape: {pred.shape}")

    mask = load_mask(mask_p, classes=classset, one_hot=False)
    mask_resized = tf.image.resize(mask[..., np.newaxis], (256, 256), method='nearest')
    mask_resized = mask_resized[..., 0].numpy().astype(np.uint8)
    print(f"Mask resized shape: {mask_resized.shape}")

    pred_one_hot = to_categorical(pred, num_classes=len(classset.classes))
    mask_one_hot = to_categorical(mask_resized, num_classes=len(classset.classes))
    print(f"Pred one-hot shape: {pred_one_hot.shape}")
    print(f"Mask one-hot shape: {mask_one_hot.shape}")

    metrics = tester.eval.evaluate(pred_one_hot, gt=mask_one_hot)
    metrics_void = tester.eval_void.evaluate(pred_one_hot, gt=mask_one_hot)

    print(f"Metrics for {img_p.name}:\n{metrics}")
    print(f"Metrics with void borders for {img_p.name}:\n{metrics_void}")
    print("-" * 50)

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(1, 256, 256, 3))
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
Pred shape: (256, 256)
Mask resized shape: (256, 256)
Pred one-hot shape: (256, 256, 7)
Mask one-hot shape: (256, 256, 7)
Metrics for test_01.jpg:
	 iou [soft]:
		 bg: 0.8588 [0.8588]
		 brt: 0.0000 [0.0000]
		 ccp/kub: 0.0692 [0.0692]
		 gl: 0.0408 [0.0408]
		 py/mrc: 0.9151 [0.9151]
		 sph: 0.7263 [0.7263]
		 tnt/ttr: 0.0000 [0.0000]
	 mean iou [soft]: 0.3729 [0.3729]
	 acc: 0.8969

Metrics with void borders for test_01.jpg:
	 iou [soft]:
		 bg: 0.8588 [0.8588]
		 brt: 0.0000 [0.0000]
		 ccp/kub: 0.0692 [0.0692]
		 gl: 0.0408 [0.0408]
		 py/mrc: 0.9151 [0.9151]
		 sph: 0.7263 [0.7263]
		 tnt/ttr: 0.0000 [0.0000]
	 mean iou [soft]: 0.3729 [0.3729]
	 acc: 0.8969

--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Pred shape: (256, 256)
Mask resized shape: (256, 256)
Pred one-hot shape: (256, 256, 7)
Mask one-hot shape: (256, 256, 7)
Metrics for test_02.jpg:
	 iou [soft]:
		 bg: 0.6540 [0.6540]
		 brt: 1.0000 [1.0